In [1]:
import os
import math
import time
import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mticker
import pandas as pd
import Scripts.fishUtilities as fu
import Scripts.fishFits as ff
import Scripts.surfaceAreaEstimators as sea

ModuleNotFoundError: No module named 'cv2'

In [ ]:
imagePath = r'/home/mc/Documents/gitRepos/fish_contours/Pictures/Robots'
databasePath = r'./'
figurePath = r'./'
saveDatabase = r'RobotFit.xlsx'
    
df = pd.read_excel(os.path.join(databasePath, 'robotSurfaceArea.xlsx'))

specimens = df['Photo Name'].tolist()
family = df['Family'].tolist()
locomotion = df['Locomotion'].tolist()
totalLengths = df['Total Length'].values
standardLengths = df['Standard Length'].values
widths = df['Width'].values
heights = df['Height'].values
mass = df['Mass'].values
surfaceAreas = df['Surface Area'].values
    
nacaSA = np.zeros(len(specimens))
polySA = np.zeros(len(specimens))
ESA = np.zeros(len(specimens))
ESAr = np.zeros(len(specimens))
ellipsoidSA = np.zeros(len(specimens))
partitionSA = np.zeros(len(specimens))
index = np.arange(0, len(specimens), 1)

In [ ]:
widthLocation = []
maxWidth = []
heightLocation = []
maxHeight = []
outAreaRatio = []
sideTopFit = []
sideBottFit = []
nacaTopFit = []
nacaBottFit = []
topTopFit = []
topBottFit = []

In [ ]:
for idx, specimen, tLength, sLength, width, height, weight in zip(index, specimens, totalLengths, standardLengths, widths, heights, mass):
    imagePathTop = fu.get_image_files(imagePath, specimen + 'TopNF.png')
    imagePathSide = fu.get_image_files(imagePath, specimen + 'SideNF.png')
    imagePathSideFull = fu.get_image_files(imagePath, specimen + 'Side.png')
    
    # import images
    imageTop = cv2.imread(imagePathTop[0])
    imageSide = cv2.imread(imagePathSide[0])
    imageSideFull = cv2.imread(imagePathSideFull[0])
    
    # make a copy of the image
    sideImage = imageSide.copy()
    topImage = imageTop.copy()
    sideImageFull = imageSideFull.copy()
        
    # get contours
    cntsTop, hullTop = fu.get_contours(imageTop)
    cntsSide, hullSide = fu.get_contours(imageSide)
    cntsFull, hullFUll = fu.get_contours(imageSideFull)
    
    # determine the most extreme points along the contour
    extSideLeft = tuple(cntsSide[cntsSide[:, :, 0].argmin()][0])
    extSideRight = tuple(cntsSide[cntsSide[:, :, 0].argmax()][0])
    extTopLeft = tuple(cntsTop[cntsTop[:, :, 0].argmin()][0])
    extTopRight = tuple(cntsTop[cntsTop[:, :, 0].argmax()][0])
    extTop = tuple(cntsFull[cntsFull[:, :, 1].argmin()][0])
    extBot = tuple(cntsFull[cntsFull[:, :, 1].argmax()][0])
    
    # Find the area ratio of fins and no fins
    areaFull = cv2.contourArea(cntsFull)
    areaNF = cv2.contourArea(cntsSide)
    areaRatio = (1-np.abs(areaNF/areaFull))
    outAreaRatio.append(areaRatio)
    
    # split the hull into top and bottom
    (cxTop, cyTop), topHullTop, bottomHullTop = fu.split_by_centroid_top(imageTop.shape, cntsTop)
    (cxSide, cySide), areaSide, topHullSide, bottomHullSide = fu.split_by_centroid_side(imageSide.shape, cntsSide)
    (cxFull, cyFull), areaSideFull, topHullSideFull, bottomHullSideFull = fu.split_by_centroid_side(imageSideFull.shape, cntsFull)

    # get contour data
    areaTop, tHullTop, bHullTop = fu.scale_data_top(topHullTop, bottomHullTop)
    scaleFactorSide, tHullSide, bHullSide = fu.scale_data(topHullSide, bottomHullSide)
    scaleFactorSideFull, tHullSideFull, bHullSideFull = fu.scale_data(topHullSideFull, bottomHullSideFull)
        
    #areaTopRatio = 1-(areaTop/areaTopFull)
    # get min and max points
    maxPointsTop, minPointsTop = fu.get_min_max(tHullTop, bHullTop)
    widthLocation.append(np.mean([minPointsTop[0],maxPointsTop[0]]))
    maxWidth.append(np.abs(minPointsTop[1]) + maxPointsTop[1])
    maxPointsSide, minPointsSide = fu.get_min_max(tHullSide, bHullSide)
    heightLocation.append(np.mean([minPointsSide[0],maxPointsSide[0]]))
    maxHeight.append(np.abs(minPointsSide[1]) + maxPointsSide[1])
        
    dx = np.linspace(0, 1.0, 200)
    # fit data
    topPolyArea, topFitTop, bottomFitTop = ff.fit_top_contours_poly(tHullTop, bHullTop, 6)
    nacaArea, m, thickness, d0, xu, yu, xl, yl = ff.fit_top_contours(areaTop, np.mean([minPointsTop[0],maxPointsTop[0]]), np.abs(minPointsTop[1])+maxPointsTop[1], (tHullTop[1,-5]+np.abs(bHullTop[1,-5]))/2.0)
    topFitSide, bottomFitSide = ff.fit_side_contours(tHullSide, bHullSide, 6)
    
    sideTopFit.append(topFitSide)
    sideBottFit.append(bottomFitSide)
    nacaTopFit.append(np.array([m, thickness, d0, 0, 0]))
    nacaBottFit.append(np.array([m, thickness, d0, 0, 0]))
    topTopFit.append(topFitTop)
    topBottFit.append(bottomFitTop)
    
    topPolyTop = np.poly1d(topFitTop)
    bottomPolyTop = np.poly1d(bottomFitTop)
    topPolySide = np.poly1d(topFitSide)
    bottomPolySide = np.poly1d(bottomFitSide)
        
    nacaSA[idx] = sea.surface_area_fusiform(topPolySide, bottomPolySide, xu, yu, sLength)
    nacaSA[idx] = nacaSA[idx] + (nacaSA[idx]*areaRatio)
    polySA[idx] = sea.surface_area_oval(dx, topPolySide, bottomPolySide, topPolyTop, bottomPolyTop, sLength)
    polySA[idx] = polySA[idx] + (polySA[idx]*areaRatio)
    _, ESA[idx], _, ESAr[idx], _ = sea.equivalentSpheroid(tLength, weight, 1025)
    ellipsoidSA[idx] = sea.ellipsoidApproximation(tLength, width, height)
    partitionSA[idx] = sea.partitionDisc(sLength, topPolySide, bottomPolySide, topPolyTop, bottomPolyTop)
    partitionSA[idx] = partitionSA[idx] + (partitionSA[idx]*areaRatio)

In [ ]:
nacaError = np.divide(np.abs(surfaceAreas-nacaSA), surfaceAreas)*100
polyError = np.divide(np.abs(surfaceAreas-polySA), surfaceAreas)*100
ESAError = np.divide(np.abs(surfaceAreas-ESA), surfaceAreas)*100
ellipsoidError = np.divide(np.abs(surfaceAreas-ellipsoidSA), surfaceAreas)*100
partitionError = np.divide(np.abs(surfaceAreas-partitionSA), surfaceAreas)*100
        
output = {'Specimen':specimens, 'Total Lengths':totalLengths, 'Standard Lengths':standardLengths,
          'ActualSA':surfaceAreas, 'NACASA':nacaSA, 'PolySA':polySA, 'ESA':ESA, 'ESA_r':ESAr, 
          'Ellipsoid':ellipsoidSA, 'Partition':partitionSA}
    
outputDF = pd.DataFrame(data=output)

In [ ]:
generalData = {'Species Name':specimens, 'Family':family, 'Locomotion':locomotion, 'Reported Length':totalLengths, 'Surface Area':surfaceAreas, 
                 'Width Location':widthLocation, 'Max Width':maxWidth, 'Height Location':heightLocation, 'Area Ratio':outAreaRatio}

outputGeneral = pd.DataFrame(data=generalData)

In [ ]:
# Compile the side data
sideFitData = {'Species Name':[], 'Family':[], 'Ta6':[], 'Ta5':[], 'Ta4':[], 'Ta3':[], 'Ta2':[], 'Ta1':[], 'Ta0':[],
              'Ba6':[], 'Ba5':[], 'Ba4':[], 'Ba3':[], 'Ba2':[], 'Ba1':[], 'Ba0':[]}
for idx, specimen in enumerate(specimens):
    sideFitData['Species Name'].append(specimen)
    sideFitData['Family'].append(family[idx])
    sideFitData['Ta6'].append(sideTopFit[idx][0])
    sideFitData['Ta5'].append(sideTopFit[idx][1])
    sideFitData['Ta4'].append(sideTopFit[idx][2])
    sideFitData['Ta3'].append(sideTopFit[idx][3])
    sideFitData['Ta2'].append(sideTopFit[idx][4])
    sideFitData['Ta1'].append(sideTopFit[idx][5])
    sideFitData['Ta0'].append(sideTopFit[idx][6])
    sideFitData['Ba6'].append(sideBottFit[idx][0])
    sideFitData['Ba5'].append(sideBottFit[idx][1])
    sideFitData['Ba4'].append(sideBottFit[idx][2])
    sideFitData['Ba3'].append(sideBottFit[idx][3])
    sideFitData['Ba2'].append(sideBottFit[idx][4])
    sideFitData['Ba1'].append(sideBottFit[idx][5])
    sideFitData['Ba0'].append(sideBottFit[idx][6])
    
outputSide = pd.DataFrame(data=sideFitData)

# Compile the top data
topFitData = {'Species Name':[], 'Family':[], 'Ta4':[], 'Ta3':[], 'Ta2':[], 'Ta1':[], 'Ta0':[],
              'Ba4':[], 'Ba3':[], 'Ba2':[], 'Ba1':[], 'Ba0':[]}
for idx, specimen in enumerate(specimens):
    topFitData['Species Name'].append(specimen)
    topFitData['Family'].append(family[idx])
    topFitData['Ta4'].append(nacaTopFit[idx][0])
    topFitData['Ta3'].append(nacaTopFit[idx][1])
    topFitData['Ta2'].append(nacaTopFit[idx][2])
    topFitData['Ta1'].append(nacaTopFit[idx][3])
    topFitData['Ta0'].append(nacaTopFit[idx][4])
    topFitData['Ba4'].append(nacaBottFit[idx][0])
    topFitData['Ba3'].append(nacaBottFit[idx][1])
    topFitData['Ba2'].append(nacaBottFit[idx][2])
    topFitData['Ba1'].append(nacaBottFit[idx][3])
    topFitData['Ba0'].append(nacaBottFit[idx][4])
    
outputTop = pd.DataFrame(data=topFitData)

In [ ]:
writer = pd.ExcelWriter(os.path.join(databasePath, saveDatabase), engine='xlsxwriter')

outputGeneral.to_excel(writer, sheet_name='General', index=False)
outputSide.to_excel(writer, sheet_name='Side Fit', index=False)
outputTop.to_excel(writer, sheet_name='Top Fit', index=False)

writer.save()

In [ ]:
mpl.rcParams['font.size'] = 18
csfont = {'fontname':'Times New Roman'}
xlabels = ['Harvard Beihang\n Mackerel', 'MIT Carangiform', 'New York University\n iDevice', 'Naval Research Lab\n 4fin UUV']
fig1, ax1 = plt.subplots(figsize=(12,10))
ax1.plot(specimens, nacaError, 'ro', label='Current Research')
ax1.plot(specimens, ESAError, 'b^', label='Murphy and Haroutunian')
ax1.plot(specimens, ellipsoidError, 'go', label='Rantung Ellipsoid')
ax1.plot(specimens, partitionError, 'gd', label='Rantung Partition')
ax1.set_ylabel('Percent Error')
#ax1.xaxis.set_major_locator(mticker.MaxNLocator(4))
#ticks_loc = ax1.get_xticks().tolist()
#ax1.xaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
#ax1.set_xticklabels([label_format.format(x) for x in ticks_loc])
ax1.set_xticklabels(xlabels, rotation=45, ha='center')
ax1.legend(bbox_to_anchor=(0.90, 1.13), ncol=2)
ax1.grid(True, which='both',linestyle='-.')
#fig1.savefig('Robot Comparison.pdf', bbox_inches='tight')

In [ ]:
xlabels = ['Harvard Beihang\n Mackerel', 'MIT Carangiform', 'New York University\n iDevice', 'Naval Research Lab\n 4fin UUV']
fig2, ax2 = plt.subplots(figsize=(12,10))
ax2.plot(specimens, nacaError, 'ro', label='NACA fit')
ax2.plot(specimens, polyError, 'rd', label='Poly fit')
ax2.set_ylabel('Percent Error')
ax2.set_xticklabels(xlabels, rotation=45, ha='center')
ax2.grid(True, which='both',linestyle='-.')
ax2.legend(bbox_to_anchor=(0.70, 1.1), ncol=2)
#fig2.savefig('Two Method Comparison.pdf', bbox_inches='tight')